In [8]:
import math
import numpy as np
from PIL import Image, ImageDraw

def check_rectangle(bounds, pixel_count):
    width = bounds["maxCol"] - bounds["minCol"] + 1
    height = bounds["maxRow"] - bounds["minRow"] + 1
    return pixel_count == width * height


def check_circle(pixel_count, bounds):
    width = bounds["maxCol"] - bounds["minCol"] + 1
    height = bounds["maxRow"] - bounds["minRow"] + 1

    if width != height:
        return False

    radius = width / 2
    expected_area = math.pi * (radius ** 2)

    return abs(pixel_count - expected_area) < expected_area * 0.35


# ---------------- Iterative DFS ----------------
def dfs_iterative(matrix, visited, sx, sy, target, bounds):
    rows, cols = matrix.shape
    stack = [(sx, sy)]
    count = 0

    while stack:
        x, y = stack.pop()

        if x < 0 or x >= rows or y < 0 or y >= cols:
            continue
        if visited[x][y] or matrix[x][y] != target:
            continue

        visited[x][y] = True
        count += 1

        bounds["minRow"] = min(bounds["minRow"], x)
        bounds["maxRow"] = max(bounds["maxRow"], x)
        bounds["minCol"] = min(bounds["minCol"], y)
        bounds["maxCol"] = max(bounds["maxCol"], y)

        stack.extend([
            (x+1, y), (x-1, y),
            (x, y+1), (x, y-1)
        ])

    return count


# ---------------- Main ----------------
def detect_objects_from_image(input_image_path, output_image_path, margin=5):
    img_gray = Image.open(input_image_path).convert("L")
    img_small = img_gray.resize((20, 20), Image.BILINEAR)

    img_np = np.array(img_small)

    threshold = img_np.mean() - margin
    binary = (img_np < threshold).astype(int)

    if binary.sum() == 0:
        print("No object detected (all background).")
        return

    rows, cols = binary.shape
    visited = np.zeros((rows, cols), dtype=bool)

    output_img = img_small.resize((200, 200), Image.NEAREST).convert("RGB")
    draw = ImageDraw.Draw(output_img)

    scale_x = 200 // 20
    scale_y = 200 // 20

    object_id = 1

    for i in range(rows):
        for j in range(cols):
            if binary[i][j] == 1 and not visited[i][j]:
                bounds = {
                    "minRow": rows,
                    "maxRow": -1,
                    "minCol": cols,
                    "maxCol": -1
                }

                pixel_count = dfs_iterative(binary, visited, i, j, 1, bounds)

                # -------- Shape Detection --------
                if check_rectangle(bounds, pixel_count):
                    shape = "RECTANGLE"
                elif check_circle(pixel_count, bounds):
                    shape = "CIRCLE"
                else:
                    shape = "UNKNOWN"

                print(f"Object {object_id}: {shape}")

                # Draw bounding box
                draw.rectangle(
                    [
                        (bounds["minCol"] * scale_x, bounds["minRow"] * scale_y),
                        ((bounds["maxCol"] + 1) * scale_x,
                         (bounds["maxRow"] + 1) * scale_y)
                    ],
                    outline="red",
                    width=2
                )

                object_id += 1

    output_img.save(output_image_path)
    print(f"Output saved as: {output_image_path}")


In [16]:
detect_objects_from_image("input1.png","output1.png")
detect_objects_from_image("input2.jpg","output2.jpg")
detect_objects_from_image("input3.png","output3.png")
detect_objects_from_image("input4.png","output4.png")
detect_objects_from_image("input5.png","output5.png")

Object 1: RECTANGLE
Output saved as: output1.png
Object 1: CIRCLE
Output saved as: output2.jpg
Object 1: CIRCLE
Output saved as: output3.png
Object 1: UNKNOWN
Output saved as: output4.png
Object 1: RECTANGLE
Output saved as: output5.png
